In [0]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
import numpy as np

Using TensorFlow backend.


In [1]:
# # Run this cell to mount your Google Drive.
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
with open('./data/shakespeare.txt', 'r') as f:
      text = f.read().lower()
print('Corpus length in characters:', len(text))

Corpus length in characters: 5445610


## Decreasing the corpus size for memory

In [0]:
text = text[:3000000]

In [0]:
chars = sorted(list(set(text)))
print('Total Number of Unique Characters:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars)) # Character to index
indices_char = dict((i, c) for c, i in char_indices.items()) # Index to Character

Total Number of Unique Characters: 57


# Creating training data

In [0]:
max_len = 50
sentences = []
next_chars = []

for i in range(len(text) - max_len):
    sentence = text[i: i + max_len]
    next_char = text[i + max_len]
    sentences.append(sentence)
    next_chars.append(next_char)

### Releasing memory

In [0]:
text = None

## Placeholders for training in a supervised environment

In [0]:
x = np.zeros((len(sentences), max_len, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

In [0]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        # Populate Tensor Input
        x[i, t, char_indices[char]] = 1 
    # Populate y with the character just after the sequence
    y[i, char_indices[next_chars[i]]] = 1

In [0]:
filepath = '/content/drive/My Drive/dataset/saved_weights_language_model.hdf5'
checkpoint = ModelCheckpoint(filepath, save_best_only=False, save_weights_only=False)
callbacks_list = [checkpoint]

### Stacked LSTM with 2 layer 

In [0]:
print('Building model...')
# Size of vector in the hidden layer.
hidden_size = 256
# Initialize Sequential Model
model = Sequential()
model.add(LSTM(hidden_size, input_shape=(max_len, len(chars)), return_sequences=True))
model.add(LSTM(hidden_size))
model.add(Dropout(0.5))
# Add the output layer that is a softmax of the number of characters
model.add(Dense(len(chars), activation='softmax')) 
# Optimization through RMSprop
optimizer_new = RMSprop() 
# Consider cross Entropy loss. Why? MLE of P(D | theta)
model.compile(loss='categorical_crossentropy', optimizer=optimizer_new) 

model.fit(x, y,
          batch_size=128,
          epochs=5, 
          validation_split=0.01)

W0710 04:00:34.734450 139881602815872 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0710 04:00:34.772166 139881602815872 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0710 04:00:34.778587 139881602815872 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



Building model...


W0710 04:00:35.695537 139881602815872 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0710 04:00:35.706248 139881602815872 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0710 04:00:35.747350 139881602815872 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0710 04:00:35.756458 139881602815872 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tenso

Train on 2969950 samples, validate on 30000 samples
Epoch 1/5
2969950/2969950 [==============================] - 3320s 1ms/step - loss: 1.6156 - val_loss: 1.3621
Epoch 2/5
2969950/2969950 [==============================] - 3325s 1ms/step - loss: 1.3724 - val_loss: 1.3098
Epoch 3/5
1800960/2969950 [=================>............] - ETA: 21:44 - loss: 1.3374Buffered data was truncated after reaching the output size limit.

In [0]:
model.save_weights('/content/drive/My Drive/dataset/saved_weights_language_model.hdf5', overwrite=True)

In [0]:
def sample(preds, temperature=1.):
    """Perform Temperature Sampling"""
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    # Softmax of predictions
    preds = exp_preds / np.sum(exp_preds) 
    # Sample a single characters, with probabilities defined in `preds`
    probas = np.random.multinomial(1, preds[0], 1) 
    return np.argmax(probas)

## Example with differnt seeds and temperature

In [25]:
sentence = sentences[111]
print('seed ====', sentence)

temperature = 0.6
for i in range(400):
    text = sentence[i: i + max_len]
    x_to_be_pred = np.zeros((1, max_len, len(chars)))
    for t, char in enumerate(text):
        x_to_be_pred[0, t, char_indices[char]] = 1.
    ans = model.predict(x_to_be_pred)
    predicted_char = indices_char[sample(ans, temperature)]
    sentence += predicted_char

print('\n Machine written paragraph \n', sentence)


seed ==== ie,
  but as the riper should by time decease,
  h

 Machine written paragraph 
 ie,
  but as the riper should by time decease,
  how known the duke of same in the captain,
    signify that the duke of clifford and i say i have no tongue
    that he a one he said your captain of herself within
    with the slack of thee, and in death endure to me do
    a country to follow the sake of the world,
    and privilent cardinal hath look his answers.
  king henry. good my lord, make me the king.
                                    


In [29]:
sentence = 'for she is the beauty of the skin that holds in my'
print('seed ====', sentence)

temperature = 1.0
for i in range(400):
    text = sentence[i: i + max_len]
    x_to_be_pred = np.zeros((1, max_len, len(chars)))
    for t, char in enumerate(text):
        x_to_be_pred[0, t, char_indices[char]] = 1.
    ans = model.predict(x_to_be_pred)
    predicted_char = indices_char[sample(ans, temperature)]
    sentence += predicted_char

print('\n Machine written paragraph \n', sentence)

seed ==== for she is the beauty of the skin that holds in my

 Machine written paragraph 
 for she is the beauty of the skin that holds in my ladies.
    a day
  and love treader resald'd him  
   itned rich peraltian moes.
  second lord. and all king shall ill lead in once carried to these,
    this time to th' king to   making fear
    of walks; shall seem the knowly ear. cave malice, further
    glory pointcred and a countrypefalle;
    and as the man of a dead bearford one
    you were the comes to prive biy, and give the world.
  


In [30]:
sentence = 'for she is the beauty of the skin that holds in my'
print('seed ====', sentence)

temperature = 2.0
for i in range(400):
    text = sentence[i: i + max_len]
    x_to_be_pred = np.zeros((1, max_len, len(chars)))
    for t, char in enumerate(text):
        x_to_be_pred[0, t, char_indices[char]] = 1.
    ans = model.predict(x_to_be_pred)
    predicted_char = indices_char[sample(ans, temperature)]
    sentence += predicted_char

print('\n Machine written paragraph \n', sentence)

seed ==== for she is the beauty of the skin that holds in my

 Machine written paragraph 
 for she is the beauty of the skin that holds in my ho't cit whithfei!cbur
  h; likeferved mry? driy, hycks,sol.
  first citizen. it' cushiot yz'orail:
  well hu bass, caa i bags;
    oo, aler; a < wess; or tuquguys-
  oph. weoi'y hemy swa. kmy oxly,
     reus;. take myhpkilr.hingdon.
  orlando.egacmbefrude's. doltelogt] vellus
 
  iardinna. , bitionmera.

  cactori]-bidst?
  riched ho,[it with vac
    byu itquepir exepegitf
   oorsqu't grestgomje


In [31]:
sentence = 'for she is the beauty of the skin that holds in my'
print('seed ====', sentence)

temperature = 0.6
for i in range(400):
    text = sentence[i: i + max_len]
    x_to_be_pred = np.zeros((1, max_len, len(chars)))
    for t, char in enumerate(text):
        x_to_be_pred[0, t, char_indices[char]] = 1.
    ans = model.predict(x_to_be_pred)
    predicted_char = indices_char[sample(ans, temperature)]
    sentence += predicted_char

print('\n Machine written paragraph \n', sentence)

seed ==== for she is the beauty of the skin that holds in my

 Machine written paragraph 
 for she is the beauty of the skin that holds in my food and sound him a place
    to make it as a good book of breath,
    and the place of impotitions of the son,
    and the great enemies that were great thing than i had remembers
    the towns to the crown age should ack the torn with him,
    i give me entertain'd the court in the man is a complexions,
    in his hand of brother the day.
  holofernes. i will not come, i would i am sure as the


In [28]:
sentence = 'for she is the beauty of the skin that holds in my'
print('seed ====', sentence)

temperature = 0.1
for i in range(400):
    text = sentence[i: i + max_len]
    x_to_be_pred = np.zeros((1, max_len, len(chars)))
    for t, char in enumerate(text):
        x_to_be_pred[0, t, char_indices[char]] = 1.
    ans = model.predict(x_to_be_pred)
    predicted_char = indices_char[sample(ans, temperature)]
    sentence += predicted_char

print('\n Machine written paragraph \n', sentence)

seed ==== for she is the beauty of the skin that holds in my

 Machine written paragraph 
 for she is the beauty of the skin that holds in my son,
    and the state of the parts of the country,
    and the country that i say the country that i say the prince of such a service
    that we will be the country than the seal of the countrymen,
    and the country that i will not be a soul
    and the first and the country and the state
    that we shall be the state of the state of the state
    and the state of the country shall be so sou
